In [148]:
import pandas as pd
import openai
import ast
from dateutil import parser
from datetime import datetime

In [30]:
openai_api_key ='your-api-key'
openai.api_key = openai_api_key

In [343]:
OPENAI_MAX_ATTEMPTS = 5

def answer_question_gpt(
    dict_prompt,
    model='text-davinci-003', # GPT-3.5
    max_tokens=1000,
    stop_sequence=None,
    temperature=0,
    top_p=0.2,
    frequency_penalty=0,
    presence_penalty=0,
):
    prompt = f"""
            {dict_prompt['system']}
            Context:\n```{dict_prompt['context']}```
    """
    
    for attempt in range(OPENAI_MAX_ATTEMPTS):
        try:
            response = openai.Completion.create(
                prompt=prompt,
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=top_p,
                frequency_penalty=frequency_penalty,
                presence_penalty=presence_penalty,
                stop=stop_sequence,
                model=model,
            )

            dict_prompt['response'] = response['choices'][0]['text'].strip()
            break
        except Exception as e:
            print(f'Attempt {attempt + 1} failed, retrying...')
            print(e)
                
    return dict_prompt

def parse_response(response):

    try:
        response = ast.literal_eval(response)
    except:
        try:
            response_json = response[response.rfind('{'): response.rfind('}')+1]
            response = ast.literal_eval(response_json)
        except Exception as e:
            print(str(e))
            pass

    return response

In [262]:
table_template = pd.read_csv('../data/template.csv', parse_dates=True)
table_template.head()

,Date,EmployeeName,Plan,PolicyNumber,Premium
0,01-05-2023,John Doe,Gold,AB12345,150
1,02-05-2023,Jane Smith,Silver,CD67890,100
2,03-05-2023,Michael Brown,Bronze,EF10111,50
3,04-05-2023,Alice Johnson,Gold,GH12121,150
4,05-05-2023,Bob Wilson,Silver,IJ13131,100


In [263]:

def get_date(s_date):
    date_patterns = ["%Y-%m-%d", "%d-%m-%Y", "%Y/%m/%d", "%m/%d/%Y", "%d/%m/%Y"]

    for pattern in date_patterns:
        try:
            dd = datetime.strptime(s_date, pattern).date()
            return pattern
        except:
            pass
        
def convert_date_column(df, pattern=None):
    date_pattern = None
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                pattern1 = get_date(str(df[col].iloc[0]))
                if pattern1: date_pattern=pattern1
                if not pattern: pattern = pattern1
                df[col] = pd.to_datetime(df[col], format=pattern1).dt.strftime(pattern)
            except ValueError:
                pass

    return df, date_pattern
    

In [264]:
table_template, date_pattern = convert_date_column(table_template)
table_template.dtypes

/var/folders/5c/36s4v3_d7y33dgvqf59bsqr80000gn/T/ipykernel_76513/1505610472.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], format=pattern1).dt.strftime(pattern)
/var/folders/5c/36s4v3_d7y33dgvqf59bsqr80000gn/T/ipykernel_76513/1505610472.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], format=pattern1).dt.strftime(pattern)
/var/folders/5c/36s4v3_d7y33dgvqf59bsqr80000gn/T/ipykernel_76513/1505610472.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], format=pattern1

Date            object
EmployeeName    object
Plan            object
PolicyNumber    object
Premium          int64
dtype: object

In [265]:
date_pattern

'%d-%m-%Y'

In [266]:
table_template

,Date,EmployeeName,Plan,PolicyNumber,Premium
0,01-05-2023,John Doe,Gold,AB12345,150
1,02-05-2023,Jane Smith,Silver,CD67890,100
2,03-05-2023,Michael Brown,Bronze,EF10111,50
3,04-05-2023,Alice Johnson,Gold,GH12121,150
4,05-05-2023,Bob Wilson,Silver,IJ13131,100
5,06-05-2023,Carol Martinez,Bronze,KL14141,50
6,07-05-2023,David Anderson,Gold,MN15151,150
7,08-05-2023,Eva Thomas,Silver,OP16161,100
8,09-05-2023,Frank Jackson,Bronze,QR17171,50
9,10-05-2023,Grace White,Gold,ST18181,150


In [414]:
table_A = pd.read_csv('../data/table_A.csv')
table_A.head()

,Date_of_Policy,FullName,Insurance_Plan,Policy_No,Monthly_Premium,Department,JobTitle,Policy_Start,Full_Name,Insurance_Type,Policy_Num,Monthly_Cost
0,05/01/2023,John Doe,Gold Plan,AB-12345,150.0,IT,Software Engineer,05/01/2023,John Doe,Gold,AB-12345,150.0
1,05/02/2023,Jane Smith,Silver Plan,CD-67890,100.0,HR,HR Manager,05/02/2023,Jane Smith,Silver,CD-67890,100.0
2,05/03/2023,Michael Brown,Bronze Plan,EF-10111,50.0,Marketing,Marketing Coordinator,05/03/2023,Michael Brown,Bronze,EF-10111,50.0
3,05/04/2023,Alice Johnson,Gold Plan,GH-12121,150.0,Finance,Financial Analyst,05/04/2023,Alice Johnson,Gold,GH-12121,150.0
4,05/05/2023,Bob Wilson,Silver Plan,IJ-13131,100.0,Sales,Sales Executive,05/05/2023,Bob Wilson,Silver,IJ-13131,100.0


In [415]:
table_A, dp = convert_date_column(table_A, date_pattern)
table_A.dtypes

/var/folders/5c/36s4v3_d7y33dgvqf59bsqr80000gn/T/ipykernel_76513/1505610472.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], format=pattern1).dt.strftime(pattern)
/var/folders/5c/36s4v3_d7y33dgvqf59bsqr80000gn/T/ipykernel_76513/1505610472.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], format=pattern1).dt.strftime(pattern)
/var/folders/5c/36s4v3_d7y33dgvqf59bsqr80000gn/T/ipykernel_76513/1505610472.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], format=pattern1

Date_of_Policy      object
FullName            object
Insurance_Plan      object
Policy_No           object
Monthly_Premium    float64
Department          object
JobTitle            object
Policy_Start        object
Full_Name           object
Insurance_Type      object
Policy_Num          object
Monthly_Cost       float64
dtype: object

In [416]:
table_A

,Date_of_Policy,FullName,Insurance_Plan,Policy_No,Monthly_Premium,Department,JobTitle,Policy_Start,Full_Name,Insurance_Type,Policy_Num,Monthly_Cost
0,05-01-2023,John Doe,Gold Plan,AB-12345,150.0,IT,Software Engineer,05-01-2023,John Doe,Gold,AB-12345,150.0
1,05-02-2023,Jane Smith,Silver Plan,CD-67890,100.0,HR,HR Manager,05-02-2023,Jane Smith,Silver,CD-67890,100.0
2,05-03-2023,Michael Brown,Bronze Plan,EF-10111,50.0,Marketing,Marketing Coordinator,05-03-2023,Michael Brown,Bronze,EF-10111,50.0
3,05-04-2023,Alice Johnson,Gold Plan,GH-12121,150.0,Finance,Financial Analyst,05-04-2023,Alice Johnson,Gold,GH-12121,150.0
4,05-05-2023,Bob Wilson,Silver Plan,IJ-13131,100.0,Sales,Sales Executive,05-05-2023,Bob Wilson,Silver,IJ-13131,100.0
5,05-06-2023,Carol Martinez,Bronze Plan,KL-14141,50.0,Operations,Operations Manager,05-06-2023,Carol Martinez,Bronze,KL-14141,50.0
6,05-07-2023,David Anderson,Gold Plan,MN-15151,150.0,Legal,Attorney,05-07-2023,David Anderson,Gold,MN-15151,150.0
7,05-08-2023,Eva Thomas,Silver Plan,OP-16161,100.0,Product,Product Manager,05-08-2023,Eva Thomas,Silver,OP-16161,100.0
8,05-09-2023,Frank Jackson,Bronze Plan,QR-17171,50.0,Engineering,Engineer,05-09-2023,Frank Jackson,Bronze,QR-17171,50.0
9,05-10-2023,Grace White,Gold Plan,ST-18181,150.0,Design,Graphic Designer,05-10-2023,Grace White,Gold,ST-18181,150.0


In [417]:
def generate_sample_dic(df):
    smp_dict = {}
    for clm in df.columns:
        smp_dict[clm] = list(df[clm].head(3).values)
        
    return smp_dict


template_dict = generate_sample_dic(table_template)
table_A_dict = generate_sample_dic(table_A)

In [418]:
template_dict

{'Date': ['01-05-2023', '02-05-2023', '03-05-2023'],
 'EmployeeName': ['John Doe', 'Jane Smith', 'Michael Brown'],
 'Plan': ['Gold', 'Silver', 'Bronze'],
 'PolicyNumber': ['AB12345', 'CD67890', 'EF10111'],
 'Premium': [150, 100, 50]}

In [419]:
template_clms = list(table_template.columns)
table_A_clms = list(table_A.columns)

In [420]:
dict_prompt = {
    'system': 'You are expert in matching table columns.',
    'context': f'There are list of two columns from two tables. \
        One is template table and the list of columns is: {template_clms}. \
        Here is an example of values in each column for the template table in the dictionary format in which the key is the column name and value is a list of sample values: \n\
            {template_dict}. \n\
        The second table we want to find the best match of columns from the template table. \
        The list of the second table columns is: {table_A_clms}. \
        Here is an example of values in each column for the second table in the dictionary format in which the key is the column name and value is a list of sample values: \n\
            {table_A_dict}. \n\
        Find a match for each column in the second table columns from template table columns based on the example values.\
        There might be many to one relationship. \n \
        Map them in a dictionary format where keys would be columns of the second table and values are the columns of the template table if best matched otherwise put None. \
        Force put value of all date related columns in the second table as the date column of the template.' + 
        '''
            A sample `Dictionary` response looks as follows:\n    
            {
                "PolicyDate": "Date",
                "Name": "EmployeeName",
                "Policy_ID": "PolicyNumber",
                "PremiumAmount": "Premium",
                "Cost": "Premium",
                'MaritalStatus': None,
            }
        '''
}
response = answer_question_gpt(dict_prompt=dict_prompt)

In [421]:
map_clms = parse_response(response['response'])
map_clms

{'Date_of_Policy': 'Date',
 'FullName': 'EmployeeName',
 'Insurance_Plan': 'Plan',
 'Policy_No': 'PolicyNumber',
 'Monthly_Premium': 'Premium',
 'Department': None,
 'JobTitle': None,
 'Policy_Start': 'Date',
 'Full_Name': 'EmployeeName',
 'Insurance_Type': 'Plan',
 'Policy_Num': 'PolicyNumber',
 'Monthly_Cost': 'Premium'}

In [422]:
def gpt_clms_mapping(df, df_template):
    clm_maps = {}
    template_clms = list(df_template.columns)
    table_clms = list(df.columns)
    table_A_dict = generate_sample_dic(df)
    template_dict = generate_sample_dic(df_template)
    
    for clm in table_clms:
        clm_values = table_A_dict[clm]
        dict_prompt = {
            'system': 'You are expert in matching table columns.',
            'context': f'There are list of two columns from two tables. \
                One is a template table and the column names are: {template_clms}. \
                Here is an example of values in each column for the template table in the dictionary format in which the key is the column name and value is a list of sample values: \n\
                    {template_dict}. \n\
                The second table we want to find the best match of a column from the template table. \
                Here is an example of values of the column {clm} for the second table: \n\
                    {clm_values}. \n\
                Find the best match for the column of the second table from template table columns based on the example values.\
                If there is not a good match response with None. \
                Fource output in the python dictionary format in which key is the `Answer` and the value is the best column name found.\
                '''
        }
        response = answer_question_gpt(dict_prompt=dict_prompt)
        
        clm_maps[clm] = parse_response(response['response'])['Answer']
        
    return clm_maps

In [423]:
columns_map = gpt_clms_mapping(table_A, table_template)
columns_map

{'Date_of_Policy': 'Date',
 'FullName': 'EmployeeName',
 'Insurance_Plan': 'Plan',
 'Policy_No': 'PolicyNumber',
 'Monthly_Premium': 'Premium',
 'Department': None,
 'JobTitle': None,
 'Policy_Start': 'Date',
 'Full_Name': 'EmployeeName',
 'Insurance_Type': 'Plan',
 'Policy_Num': 'PolicyNumber',
 'Monthly_Cost': 'Premium'}

In [424]:
def parse_name_prompt(values):
    dict_prompt = {
            'system': 'You are expert in detecting first and last name.',
            'context': f'Based on the values bellow, find if all of them `first name`, `last name`, or `first and last name`. \n \
                    Values: {values} \n \
                    Fource output in the python dictionary format with one key in which key is the `Answer` and the value is \
                    one of these [`first name`, `last name`, or `first and last name`]'
    }
    response = answer_question_gpt(dict_prompt=dict_prompt)
    
    return parse_response(response['response'])['Answer']

def parse_mapping(columns_map, table_A):
    name_values = {}
    df = pd.DataFrame()
    columns = []
    for key, value in columns_map.items():
        if value is None:
            df[key] = table_A[key]
        else:
            if value == 'EmployeeName':
                    response = parse_name_prompt(table_A.head(5)[key].values)
                    name_values[response] = table_A[key]
            elif value not in df.columns:
                df[value] = table_A[key]
                
        if 'first and last name' in name_values:
            df['EmployeeName'] = name_values['first and last name']
        elif 'first name' in name_values and 'last name' in name_values:
            df['EmployeeName'] = name_values['first name'] + ' ' + name_values['last name']
        elif 'first name' in name_values:
            df['EmployeeName'] = name_values['first name']
        elif 'last name' in name_values:
            df['EmployeeName'] = name_values['last name']
            
    return df

In [425]:
new_table = parse_mapping(columns_map, table_A)
new_table

,Date,EmployeeName,Plan,PolicyNumber,Premium,Department,JobTitle
0,05-01-2023,John Doe,Gold Plan,AB-12345,150.0,IT,Software Engineer
1,05-02-2023,Jane Smith,Silver Plan,CD-67890,100.0,HR,HR Manager
2,05-03-2023,Michael Brown,Bronze Plan,EF-10111,50.0,Marketing,Marketing Coordinator
3,05-04-2023,Alice Johnson,Gold Plan,GH-12121,150.0,Finance,Financial Analyst
4,05-05-2023,Bob Wilson,Silver Plan,IJ-13131,100.0,Sales,Sales Executive
5,05-06-2023,Carol Martinez,Bronze Plan,KL-14141,50.0,Operations,Operations Manager
6,05-07-2023,David Anderson,Gold Plan,MN-15151,150.0,Legal,Attorney
7,05-08-2023,Eva Thomas,Silver Plan,OP-16161,100.0,Product,Product Manager
8,05-09-2023,Frank Jackson,Bronze Plan,QR-17171,50.0,Engineering,Engineer
9,05-10-2023,Grace White,Gold Plan,ST-18181,150.0,Design,Graphic Designer


In [426]:
table_A.head(5)

,Date_of_Policy,FullName,Insurance_Plan,Policy_No,Monthly_Premium,Department,JobTitle,Policy_Start,Full_Name,Insurance_Type,Policy_Num,Monthly_Cost
0,05-01-2023,John Doe,Gold Plan,AB-12345,150.0,IT,Software Engineer,05-01-2023,John Doe,Gold,AB-12345,150.0
1,05-02-2023,Jane Smith,Silver Plan,CD-67890,100.0,HR,HR Manager,05-02-2023,Jane Smith,Silver,CD-67890,100.0
2,05-03-2023,Michael Brown,Bronze Plan,EF-10111,50.0,Marketing,Marketing Coordinator,05-03-2023,Michael Brown,Bronze,EF-10111,50.0
3,05-04-2023,Alice Johnson,Gold Plan,GH-12121,150.0,Finance,Financial Analyst,05-04-2023,Alice Johnson,Gold,GH-12121,150.0
4,05-05-2023,Bob Wilson,Silver Plan,IJ-13131,100.0,Sales,Sales Executive,05-05-2023,Bob Wilson,Silver,IJ-13131,100.0


In [427]:
def match_columns_format(df1, df2):
    for value in table_template.columns:
        if value is not None:
            try:
                print(f'Column {value}')
                df2[value]=df2[value].astype(df1[value].dtypes.name)
                print('-- done!')
            except Exception as err:
                print(f'-- There is an error: {err}')
                pass
            
    return df2

In [430]:
new_table = match_columns_format(table_template, new_table)

Column Date
-- done!
Column EmployeeName
-- done!
Column Plan
-- done!
Column PolicyNumber
-- done!
Column Premium
-- done!


In [459]:
new_table['Plan'].head(3).values.

['Gold Plan', 'Silver Plan', 'Bronze Plan']

In [466]:
row_list = []
for key, val in columns_map.items():
    clm_name = ['target_name', 'map_name', 'target_samples', 'map_samples']
    row = [key, val, str(table_A[key].head(3).values.tolist()),
            str(table_template[val].head(3).values.tolist()) if val else None]
    row_list.append(row)

df_train = pd.DataFrame(row_list, columns=clm_name)

In [467]:
df_train

,target_name,map_name,target_samples,map_samples
0,Date_of_Policy,Date,"['05-01-2023', '05-02-2023', '05-03-2023']","['01-05-2023', '02-05-2023', '03-05-2023']"
1,FullName,EmployeeName,"['John Doe', 'Jane Smith', 'Michael Brown']","['John Doe', 'Jane Smith', 'Michael Brown']"
2,Insurance_Plan,Plan,"['Gold Plan', 'Silver Plan', 'Bronze Plan']","['Gold', 'Silver', 'Bronze']"
3,Policy_No,PolicyNumber,"['AB-12345', 'CD-67890', 'EF-10111']","['AB12345', 'CD67890', 'EF10111']"
4,Monthly_Premium,Premium,"[150.0, 100.0, 50.0]","[150, 100, 50]"
5,Department,None,"['IT', 'HR', 'Marketing']",None
6,JobTitle,None,"['Software Engineer', 'HR Manager', 'Marketing...",None
7,Policy_Start,Date,"['05-01-2023', '05-02-2023', '05-03-2023']","['01-05-2023', '02-05-2023', '03-05-2023']"
8,Full_Name,EmployeeName,"['John Doe', 'Jane Smith', 'Michael Brown']","['John Doe', 'Jane Smith', 'Michael Brown']"
9,Insurance_Type,Plan,"['Gold', 'Silver', 'Bronze']","['Gold', 'Silver', 'Bronze']"
